In [1]:
import numpy as np
from adapt.instance_based import TrAdaBoostR2
import json
from scikeras.wrappers import KerasRegressor
from src.model import CNN
from src.util import timestamp
import keras
from tqdm import tqdm
import joblib
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, concatenate

Domainadaptation mit TradaBoostR2 aus der Adapt Libary
Erste Version, wenns läuft dann später in den Klassen die einzelnen Methoden ergänzen

---
## **Trainingsdaten erzeugen**
Da die Zuordnung der Datenmengen über die Anzahl der Versuche läuft, wird hier nur der Belchsplit durchgeführt, es wird eine bestimmte Anzahl an Blechen gewählt. 

- Trainingsdaten aus den Simulationsdaten werden wie gehabt erzeugt
- Target Domain Daten werden aus den Raldaten erzeugt.   

In [ ]:
from src.execution import WindowSplittingExecution


Simulationsdaten: Blechsplit

In [ ]:
DATA: str = "assets/synthetic-data.csv"
VALIDATION_SPLIT: bool = True # If the data should also be splitted into a validation set?
TEST_SIZE: float = 0.5
BATCH_SPLIT: bool = False
BATCHSIZE: int = 326 # real-data: 1800, synthetic-data: 326
SEED: int = 69 # Seed for random state -> Split with same seed and data will always result in the same split
INTERPOLATION: bool = False
WINDOWSIZE: int = 10
SEP: str = ";" # Separator of the csv file
DECIMAL: str = "." # Decimal separator of the csv file


WindowSplittingExecution.execute(DATA, 
                                 BATCH_SPLIT, 
                                 VALIDATION_SPLIT, 
                                 TEST_SIZE, 
                                 SEED, 
                                 BATCHSIZE, 
                                 INTERPOLATION, 
                                 WINDOWSIZE, 
                                 SEP, 
                                 DECIMAL)



Realdaten Blechsplit (`Train_Test_Split = 2`):
- Die Daten trennen in A % Trainingsdaten für den TradaBoostR2 und B % Testdaten
--> Solche Auftielung mti den Vorhandneen Methoden leichter umzusetzen als über exakte Anzahl der Bleche
- Die Domain Adaptation wird auf dem A % Testdaten durchgeführt
- Testen erfolgt auf dem Testdatensatz
- Die Größenverhältnisse der Datensätze werden über die Variable  `size` eingestellt
- `size = 0.9` so werden 10 % der Bleche in das TradaBoost DaomainAdaptaion gegeben, die restlichen 90 % der Daten werden zum Kontrollieren verwndet  

In [ ]:
DATA: str = "assets/real-data.csv"
VALIDATION_SPLIT: bool = True # If the data should also be splitted into a validation set?
TEST_SIZE: float = 0.9
BATCH_SPLIT: bool = False
BATCHSIZE: int = 1800 # real-data: 1800, synthetic-data: 326
SEED: int = 69 # Seed for random state -> Split with same seed and data will always result in the same split
INTERPOLATION: bool = False
WINDOWSIZE: int = 10
SEP: str = ";" # Separator of the csv file
DECIMAL: str = "," # Decimal separator of the csv file


WindowSplittingExecution.execute(DATA, 
                                 BATCH_SPLIT, 
                                 VALIDATION_SPLIT, 
                                 TEST_SIZE, 
                                 SEED, 
                                 BATCHSIZE, 
                                 INTERPOLATION, 
                                 WINDOWSIZE, 
                                 SEP, 
                                 DECIMAL)



## **Kontrolle der Verteilungen**
Verglichen werden die Verteilungen der Inputs und Outputs vor dem Skalierung und danach
Problem: Die Daten der Targetdomain und der Sourcdomain werdne mit unterschiedlichen Scalern skaliert, die skalierten Daten leigen immer übereinander, da mit min max scaliert wird 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd

dateipfad_femDaten="build\\window_split\\synthetic-data\\1742837410"
dateipfad_realDaten="build\\window_split\\real-data\\1742838313"

def plotAllHistOfFIle(file_name):
    # Lade die Daten aus beiden Ordnern
    fem_path = os.path.join(dateipfad_femDaten, file_name)
    real_path = os.path.join(dateipfad_realDaten, file_name)

    # Daten als numpy-Array laden
    fem_data = np.load(fem_path)
    real_data = np.load(real_path)
    fem_data = fem_data.reshape(-1, fem_data.shape[2])  # Shape: (287560, 11)
    real_data = real_data.reshape(-1, real_data.shape[2])  # Shape: (254180, 11)
    # Anzahl der Features (Spalten)
    num_features = fem_data.shape[1]
    print(fem_data.shape)
    print(real_data.shape)
    bins = 50  # Anzahl der Bins für die Histogramme


    # Anzahl der Spalten pro Zeile
    cols = 4
    rows = (num_features + cols - 1) // cols  # Rundet auf, falls nicht durch 4 teilbar

    # Plots erstellen
    fig, axes = plt.subplots(rows, cols, figsize=(20, 5 * rows))
    axes = axes.flatten()  # 2D Array in 1D umwandeln für einfachere Iteration

    for i in range(num_features):
        ax = axes[i]
        sns.histplot(fem_data[:, i], bins=bins, kde=True, ax=axes[i], color="blue", label="Simulation", stat="density", alpha=0.6)
        sns.histplot(real_data[:, i], bins=bins, kde=True, ax=axes[i], color="orange", label="Realdaten", stat="density", alpha=0.6)
        ax.set_title(f"Feature {i+1}")
        ax.set_xlabel("Wert")
        ax.set_ylabel("Dichte")
        ax.legend()

    # Leere Subplots deaktivieren, falls es weniger als 4*n Features gibt
    for j in range(i+1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()

# Datei laden



In [ ]:
plotAllHistOfFIle("x-train-scaled.npy")

In [ ]:
plotAllHistOfFIle("y-train-scaled.npy")

## **TradaBoostR2**


In [ ]:
def domainAdapt_tradaBoostR2_saving(model_file, save_folder, save_filename, learning_rate,X_source_scaled,y_source_scaled,X_target_scaled,y_target_scaled):
    print("Modell laden")
    cnn = CNN.from_file(model_file)
    base_output = cnn.model.get_layer("dense_4").output  # Falls 'dense_4' die letzte gemeinsame Schicht ist
    new_output = Dense(3, activation="linear", name="output")(base_output)
    cnn.model = Model(inputs=cnn.model.input, outputs=new_output)

    cnn.model.summary()

    #Basis Modell erzeugen
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = ['mean_absolute_error', 'mean_absolute_error', 'mean_absolute_error']
    metrics={'Verstellweg_X': 'mae', 'Verstellweg_Y': 'mae', 'Verstellweg_Phi': 'mae'}
    loss_one_output=['mean_absolute_error']
    metrics_one_output=['mae']
    cnn.model.compile(optimizer=optimizer, loss=loss_one_output,metrics=metrics_one_output)
    
    
    #Wrappen des MOdells vom Tensorflow in in Keras
    cnn_wrapped = KerasRegressor(model=cnn.model, epochs=2, batch_size=32, verbose=1)#,target_type="continuous-multioutput")#, multi_output=True)#,validate=False)
    tradaboost_model = TrAdaBoostR2(
        estimator=cnn_wrapped,  # CNN als Basis-Regressor
        n_estimators=1  # Anzahl der Boosting-Iterationen
    )
    
    y_source_scaled=np.squeeze(y_source_scaled)
    y_target_scaled=np.squeeze(y_target_scaled)
    print("\n Training mit TrAdaBoostR2 gestartet \n")
    print(f"save_folder = {save_folder}")
    print(f"")
    with tqdm(total=tradaboost_model.n_estimators, desc="TrAdaBoostR2 Fortschritt") as pbar:
        for i in range(tradaboost_model.n_estimators):
            tradaboost_model.fit(X_source_scaled, y_source_scaled,
                                    X_target_scaled, y_target_scaled
                                    )
                
                # Speichern des Zwischenstands nach jeder Iteration
            joblib.dump(tradaboost_model, os.path.join(save_folder, f"tradaboost_model_iter_{i}"))

            # Optional: Wichtige Variablen separat speichern
            checkpoint_data = {
                "iteration": i + 1,
                "estimator_count": len(tradaboost_model.estimators_),
                "weights": tradaboost_model.estimator_weights_,
                "errors": tradaboost_model.estimator_errors_,
            }
            joblib.dump(checkpoint_data, os.path.join(save_folder, f"checkpoint_data_{i}.pkl"))

            pbar.update(1)
    print("\n Training mit TrAdaBoostR2 beendet \n")
    
    # Extrahiertes Modell nach dem Training speichern
    trained_model = tradaboost_model.estimators_[-1]  # Letzter trainierter Estimator

    # Modell speichern
    trained_model.model_.save(os.path.join(save_folder, f"{save_filename}.h5"))
        
    metadata = {
        "model": model_file,
        "data": "Einzeln übergeben weil TradaBoost!",
        "optimizer": {
            "name": optimizer.name,
            "learning_rate": learning_rate
        },
        "loss": loss,
        "metrics": metrics
    }
    #TODO: Den Speicher Zeug noch hinzufügen um das Training zu dokumentieren


In [ ]:
from adapt.instance_based import TrAdaBoostR2
import json
from src.model import CNN
from src.util import timestamp
import keras
from src.data import NPY

dateipfad_femDaten="build\\window_split\\synthetic-data\\1742837410\\"
dateipfad_realDaten="build\\window_split\\real-data\\1742838313\\"
'''
#Simulationsdaten      
X_source_scaled=np.load(dateipfad_femDaten+"x-train-scaled.npy")
y_source_scaled=np.load(dateipfad_femDaten+"y-train-scaled.npy")
# Realdaten
X_target_scaled=np.load(dateipfad_realDaten+"x-train-scaled.npy")
y_target_scaled=np.load(dateipfad_realDaten+"y-train-scaled.npy")
'''
X_source_scaled = NPY.from_file(dateipfad_femDaten + "x-train-scaled.npy").array
y_source_scaled = NPY.from_file(dateipfad_femDaten + "y-train-scaled.npy").array

# Realdaten
X_target_scaled = NPY.from_file(dateipfad_realDaten + "x-train-scaled.npy").array
y_target_scaled = NPY.from_file(dateipfad_realDaten + "y-train-scaled.npy").array



#Modelldaten
model_file = "assets\\tuned-synthetic-data-offset\\1742400473\\best-model.h5"
save_folder="build\\tradaboost_model\\"
save_filename="test_training"

# Daomain Adaptation Training mit TradaBoostR2
domainAdapt_tradaBoostR2_saving(model_file=model_file,save_folder=save_folder,save_filename=save_filename,learning_rate=0.00011858185678151421,
                         X_source_scaled=X_source_scaled,y_source_scaled=y_source_scaled,X_target_scaled=X_target_scaled,y_target_scaled=y_target_scaled)



Early stopping und Valiadation hinzufügen

In [ ]:
# Validation data muss mit erzeugt werden
DATA: str = "assets/synthetic-data.csv"
VALIDATION_SPLIT: bool = True # If the data should also be splitted into a validation set?
TEST_SIZE: float = 0.5
BATCH_SPLIT: bool = False
BATCHSIZE: int = 326 # real-data: 1800, synthetic-data: 326
SEED: int = 69 # Seed for random state -> Split with same seed and data will always result in the same split
INTERPOLATION: bool = False
WINDOWSIZE: int = 10
SEP: str = ";" # Separator of the csv file
DECIMAL: str = "." # Decimal separator of the csv file


WindowSplittingExecution.execute(DATA, 
                                 BATCH_SPLIT, 
                                 VALIDATION_SPLIT, 
                                 TEST_SIZE, 
                                 SEED, 
                                 BATCHSIZE, 
                                 INTERPOLATION, 
                                 WINDOWSIZE, 
                                 SEP, 
                                 DECIMAL)

In [ ]:

def domainAdapt_tradaBoostR2_earlystopping(model_file, save_folder, save_filename, learning_rate,X_source_scaled,y_source_scaled,X_target_scaled,y_target_scaled):
    print("Modell laden")
    cnn = CNN.from_file(model_file)
    base_output = cnn.model.get_layer("dense_4").output  # Falls 'dense_4' die letzte gemeinsame Schicht ist
    new_output = Dense(3, activation="linear", name="output")(base_output)
    cnn.model = Model(inputs=cnn.model.input, outputs=new_output)

    cnn.model.summary()

    #Basis Modell erzeugen
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = ['mean_absolute_error', 'mean_absolute_error', 'mean_absolute_error']
    metrics={'Verstellweg_X': 'mae', 'Verstellweg_Y': 'mae', 'Verstellweg_Phi': 'mae'}
    loss_one_output=['mean_absolute_error']
    metrics_one_output=['mae']
    cnn.model.compile(optimizer=optimizer, loss=loss_one_output,metrics=metrics_one_output)
    
    early_stopping =keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    
    #Wrappen des MOdells vom Tensorflow in in Keras
    cnn_wrapped = KerasRegressor(model=cnn.model, epochs=2, batch_size=32, verbose=1,callbacks=[early_stopping])#,target_type="continuous-multioutput")#, multi_output=True)#,validate=False)
    tradaboost_model = TrAdaBoostR2(
        estimator=cnn_wrapped,  # CNN als Basis-Regressor
        n_estimators=1  # Anzahl der Boosting-Iterationen
    )
    
    y_source_scaled=np.squeeze(y_source_scaled)
    y_target_scaled=np.squeeze(y_target_scaled)
    print("\n Training mit TrAdaBoostR2 gestartet \n")
    print(f"save_folder = {save_folder}")
    print(f"")
    with tqdm(total=tradaboost_model.n_estimators, desc="TrAdaBoostR2 Fortschritt") as pbar:
        for i in range(tradaboost_model.n_estimators):
            tradaboost_model.fit(X_source_scaled, y_source_scaled,
                                 X_target_scaled, y_target_scaled,
                                 eval_set)
                
                # Speichern des Zwischenstands nach jeder Iteration
            joblib.dump(tradaboost_model, os.path.join(save_folder, f"tradaboost_model_iter_{i}"))

            # Optional: Wichtige Variablen separat speichern
            checkpoint_data = {
                "iteration": i + 1,
                "estimator_count": len(tradaboost_model.estimators_),
                "weights": tradaboost_model.estimator_weights_,
                "errors": tradaboost_model.estimator_errors_,
            }
            joblib.dump(checkpoint_data, os.path.join(save_folder, f"checkpoint_data_{i}.pkl"))

            pbar.update(1)
    print("\n Training mit TrAdaBoostR2 beendet \n")
    
    # Extrahiertes Modell nach dem Training speichern
    trained_model = tradaboost_model.estimators_[-1]  # Letzter trainierter Estimator

    # Modell speichern
    trained_model.model_.save(os.path.join(save_folder, f"{save_filename}.h5"))
        

    metadata = {
        "model": model_file,
        "data": "Einzeln übergeben weil TradaBoost!",
        "optimizer": {
            "name": optimizer.name,
            "learning_rate": learning_rate
        },
        "loss": loss,
        "metrics": metrics
    }